# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [32]:
# imports
from openai import OpenAI
import os
from dotenv import load_dotenv
import requests
import json
from IPython.display import Markdown, display, clear_output


In [5]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [9]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')



In [10]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why. Use markdowns.
yield from {book.get("author") for book in books if book.get("author")}
"""

In [19]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()

for chunk in openai.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that explains code."},
        {"role": "user", "content": question}
    ],
    stream=True
):
    if chunk.choices[0].delta.content:
        # Buffer to accumulate markdown output and print only on newlines
        if 'markdown_buffer' not in globals():
            markdown_buffer = ""
        markdown_buffer += chunk.choices[0].delta.content
        while "\n" in markdown_buffer:
            line, markdown_buffer = markdown_buffer.split("\n", 1)
            display(Markdown(line))
    # Print any remaining buffer at the end (outside the loop, after streaming)


In this example, the generator produced by the code would yield "Author A" and "Author B" when iterated.Certainly! Let's break down what the code does step-by-step.

### Code Breakdown

```python

yield from {book.get("author") for book in books if book.get("author")}

```

...

```

When you run the code above within a `yield` function, it would yield:

- "Author A"

- "Author B"

- "Author C"

In [33]:
# Get Llama 3.2 to answer

# Make sure Ollama is running with: ollama run llama3.2
response = requests.post('http://localhost:11434/api/generate', json={
    'model': 'llama3.2',
    'prompt': f"""You are a helpful assistant that explains code.

User question: {question}

Please provide a clear explanation:""",
    'stream': True
})

# Buffer to collect the full response
full_response = ""

for line in response.iter_lines():
    if line:
        data = json.loads(line.decode('utf-8'))
        if 'response' in data:
            full_response += data['response']
            
            # Clear previous output and display updated markdown
            clear_output(wait=True)
            display(Markdown(full_response))
            
        if data.get('done', False):
            break


**Code Explanation**
======================

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Breakdown:

*   `yield from`: This is a syntax feature introduced in Python 3.3 that allows us to delegate a sub-expression to a generator.
*   `{book.get("author") for book in books if book.get("author")}`: This is a **set comprehension**, which creates an iterable set.

**How it Works:**
-----------------

1.  `books` is assumed to be some kind of iterable (e.g., list, dictionary, or generator).
2.  For each item in `books`, the code checks if that item has a key called `"author"`.
3.  If the item does have an `"author"` key, its value is included in the iteration.
4.  The set comprehension creates a new iterable containing only the values from the items where `"author"` exists.
5.  Finally, `yield from` takes this iterable and yields each of its elements one by one.

**What it Does:**
-----------------

This code generates an iterator that yields the author names from the books in the `books` collection. It filters out any books without authors to avoid attempting to access non-existent keys.

### Example Use Case:

Suppose we have a list of book dictionaries:
```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Author C"}
]
```
Running the original code with `books` would yield:
```python
["Author A", "Author C"]
```
This demonstrates how the set comprehension and `yield from` clause work together to efficiently generate a filtered iterator of author names.